In [1]:
pip install kmapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import kmapper as km
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from datetime import datetime

# Cargar los datos desde el archivo CSV
file_path = 'covid19_data_spain_ministerio_sanidad_ACUMULADO_casos.csv'
#file_path = 'covid19_data_spain_ministerio_sanidad_ACUMULADO_fallecidos.csv'
data = pd.read_csv(file_path)
data = data.reset_index(drop=True)

# Filtrar datos para una fecha específica (Fecha == diferencia_dias)
#data = data.query('Fecha == 45')

# Crear un objeto Mapper
mapper = km.KeplerMapper(verbose=1)

# Normalizar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[['Casos', 'Latitude', 'Longitude']])
#data_scaled = scaler.fit_transform(data[['Fallecidos', 'Latitude', 'Longitude']])


KeplerMapper(verbose=1)


In [6]:
data_scaled

array([[-0.56734994, -0.6697384 , -0.228538  ],
       [-0.56734994,  0.45556243,  0.72721018],
       [-0.56734994,  0.99461071, -0.56924175],
       ...,
       [-0.28705913,  0.70708899,  0.29684387],
       [-0.50531877, -1.07251478, -0.42249466],
       [-0.50601324, -1.24011198,  0.17968827]])

In [7]:
# Proyección
lens = mapper.project(data_scaled, projection=[0, 1])

# Cubicaje y Clusterización
graph_casos = mapper.map(lens,
                         data_scaled,
                         cover=km.Cover(n_cubes=8, perc_overlap=0.10),
                         clusterer=DBSCAN(eps=0.5, min_samples=3))

# Crear color_values basado en el número de casos
color_values = data[['Casos']].values
#color_values = data[['Fallecidos']].values

# Nombre de fichero con ID
fileID = 'projection=' + graph_casos['meta_data']['projection'].split('(')[0] + '_' + \
'n_cubes=' + str(graph_casos['meta_data']['n_cubes']) + '_' + \
'perc_overlap=' + str(graph_casos['meta_data']['perc_overlap']) + '_' + \
'clusterer=' + graph_casos['meta_data']['clusterer'].split('(')[0] + '_' + \
'scaler=' + graph_casos['meta_data']['scaler'].split('(')[0]

# Obtener la fecha y hora actual
now = datetime.now()
date_str = now.strftime('%Y_%m_%d_%H_%M')

# Agregar la fecha a la cadena
fileID += '_' + date_str

# Convertir la columna 'Fecha' a fechas reales y agregar a custom_tooltips
start_date = pd.Timestamp('2020-01-01')
data['Fecha'] = start_date + pd.to_timedelta(data['Fecha'], unit='D')
custom_tooltips = data['Fecha'].dt.strftime('%Y-%m-%d') + ", " + data['CCAA']

# Visualizar el grafo de redes con etiquetas de cluster y coloración por número de casos
mapper.visualize(
    graph_casos,
    path_html="mapper_visualization_" + fileID + ".html",
    title="COVID-19 Mapper with Clustering",
    color_values=color_values,
    color_function_name=['Casos'],
    #color_function_name=['Fallecidos'],
    #node_color_function=np.mean,
    custom_tooltips=custom_tooltips
)

print("El análisis topológico y la clusterización se han completado. Se ha generado el archivo 'mapper_visualization_" + fileID + ".html.")

# Crear una función que indique qué CCAA están involucradas en cada nodo o estructura reticular
def get_ccaa_per_node(cluster_ids, data):
    ccaa_per_node = {}
    for node_id, members in cluster_ids.items():
        ccaa_in_node = data.loc[members, 'CCAA'].unique()
        ccaa_per_node[node_id] = ccaa_in_node
    return ccaa_per_node

ccaa_per_node = get_ccaa_per_node(graph_casos['nodes'], data)

# Utilizar las funciones max y min para determinar el primer y último día de los datos que abarca cada nodo.
def get_dates_per_node(cluster_ids, data):
    dates_per_node = {}
    for node_id, members in cluster_ids.items():
        dates_in_node = data.loc[members, 'Fecha']
        dates_per_node[node_id] = (dates_in_node.min(), dates_in_node.max())
    return dates_per_node

dates_per_node = get_dates_per_node(graph_casos['nodes'], data)

# Mostrar las CCAA y fechas para cada nodo
for node_id in ccaa_per_node:
    print(f"Nodo {node_id}:")
    print(f"  CCAA: {', '.join(ccaa_per_node[node_id])}")
    print(f"  Fechas: {dates_per_node[node_id][0]} a {dates_per_node[node_id][1]}")

..Projecting on data shaped (17005, 3)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (17005, 3) using lens shaped (17005, 2)

Creating 64 hypercubes.

Created 36 edges and 44 nodes in 0:00:00.673613.
Wrote visualization to: mapper_visualization_projection=[0, 1]_n_cubes=8_perc_overlap=0.1_clusterer=DBSCAN_scaler=MinMaxScaler_2024_07_28_12_50.html
El análisis topológico y la clusterización se han completado. Se ha generado el archivo 'mapper_visualization_projection=[0, 1]_n_cubes=8_perc_overlap=0.1_clusterer=DBSCAN_scaler=MinMaxScaler_2024_07_28_12_50.html.
Nodo cube0_cluster0:
  CCAA: Canarias
  Fechas: 2020-01-01 00:00:00 a 2022-03-22 00:00:00
Nodo cube1_cluster0:
  CCAA: Ceuta
  Fechas: 2020-01-01 00:00:00 a 2022-06-13 00:00:00
Nodo cube1_cluster1:
  CCAA: Melilla
  Fechas: 2020-01-01 00:00:00 a 2022-06-13 00:00:00
Nodo cube2_cluster0:
  CCAA: Andalucía, Ceuta
  Fechas: 2020-01-01 00:00:00 a 2022-06-13 00:00:00
Nodo cube3_cluster0:
  CCAA: 

In [ ]:
data['Casos'].values

array([0, 0, 0, 1, 1, 0, 0, 0, 3, 0, 0, 1, 2, 1, 0, 0, 1, 0, 0])